<a href="https://colab.research.google.com/github/japanipsystem/Test/blob/master/AT_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#機械学習を使った株価予想
Copyright (c) 2024 Takanori Adachi. All rights reserved.

**参考文献**

1. Francois Chollet, "[Deep Learning with Python, 2nd Ed.](https://www.amazon.co.jp/Learning-Python-Second-François-Chollet/dp/1617296864/ref=sr_1_1?__mk_ja_JP=カタカナ&crid=3S64D6OAIA354&keywords=deep+learning+with+python&qid=1653014306&sprefix=deep+learning+with+python%2Caps%2C165&sr=8-1)", Manning Pub. (2021).
(Kerasの開発者が著したこの本は，深層学習の標準的教科書です．)

1.  [Keras API reference](https://keras.io/api/).
(Kerasのアプリケーション・インターフェイス (API) を解説しています．
実例も豊富です．)

1. [Keras GitHub](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/python/keras).
(上記API解説文書には載っていない細かい仕様を調べようとすると，Kerasのソースコードにあたるしかありません．
このGitHubはそのときに参照する場所です．)

1. [How to Predict Stock Prices in Python using TensorFlow 2 and Keras](https://www.thepythoncode.com/article/stock-price-prediction-in-python-using-tensorflow-2-and-keras).
(Kerasを使って株価予測をしようという試みで，この講義資料執筆でも大いに参考にしました．)

1. [Yohoo_fin Documentation](http://theautomatic.net/yahoo_fin-documentation/).
(本講義で利用するデータ・ソースである yahoo_fin の標準文書です．)

1. David M. Beazley, "[Python Distilled](https://www.amazon.co.jp/Python-Essential-Reference-Developers-Library/dp/0134173279/ref=sr_1_1?__mk_ja_JP=カタカナ&crid=3JW4WHS94JOWC&keywords=Python+Distilled&qid=1653023273&s=english-books&sprefix=python+distille%2Cenglish-books%2C172&sr=1-1)", Addison Wesley
(2021).
(足立が利用している Python の教科書のなかでは，もっとも優れていると思います．)

1. David Amos, "[Object-Oriented Programming (OOP) in Python 3](https://realpython.com/python3-object-oriented-programming/)"
(手っ取り早く「オブジェクト指向プログラミング」を学びたいときに便利なサイトです．)

1. Kuroiku, [深層学習シリーズ](https://qiita.com/kuroitu/items/221e8c477ffdd0774b6b#深層学習シリーズ).
(ライブラリを使わずに，深層学習を順番に考えスクラッチから作る Python コードで解説しようとしています．)

1. Aurelien Geron, "[Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow (2nd ed.)](https://www.amazon.co.jp/gp/product/1492032646/ref=ppx_yo_dt_b_asin_title_o01_s00?ie=UTF8&psc=1)", O'Reilly 2019.
(深層学習に限らず機械学習一般について詳しく網羅された，定評のある教科書です．)

1. 大槻兼資, 秋葉拓哉, "[アルゴリズムとデータ構造](https://www.amazon.co.jp/gp/product/4065128447/ref=ppx_yo_dt_b_asin_title_o01_s00?ie=UTF8&psc=1)", 講談社 2020.
(プログラミング技術を向上させたいならば，アルゴリズムとデータ構造の学習は必須と思います．この本は内容も豊富で解説も平易です．ただし，Python ではなく C++ を使っています．)

**1. データの前処理**


最初に，日次株価データを取り込むために，ライブラリ yahoo_fin をインストールします．

In [ ]:
pip install yahoo_fin

たとえば Google（GOOGL)の日次データを取り込んでみると，以下のようになります．

In [ ]:
from yahoo_fin import stock_info as si
df = si.get_data('GOOGL')
df

各レコードは7つのカラムから成っているのがわかるでしょう．このうち終値を示す特徴量としては，株式分割や配当分配を考慮した adjusted close を利用します．

**課題**
Ticker を変更して（例えば "AAPL", "META", "AMZN" など），データを取り込んでみよ．

まずは，株価データを格納するクラス SPData を実装するために必要なライブラリを import します．

In [ ]:
from sklearn import preprocessing
from collections import deque
import numpy as np
import os
import time

Ticker を指定して，Yahoo Finance から読み込んだデータは，クラス SPData のインスタンスとして管理されます．

SPData は以下のとおりです．

In [ ]:
# individual stock price data class
class SPData(object):
    def __init__(self, ticker):
        """
        Loads data from Yahoo Finance source, as well as scaling.
        Params:
            ticker (str/pd.DataFrame): the ticker you want to load, examples include AAPL, TESL, etc.
        """

        # save parameters
        self.ticker = ticker

        # the list of features to use to feed into the model, default is everything grabbed from yahoo_fin
        self.feature_columns = ['adjclose', 'volume', 'open', 'high', 'low']

        # load data from Yahoo Finance source
        self.df = self.load_df()

    def load_df(self): # load data from Yahoo Finance source
        df = si.get_data(self.ticker) # load from yahoo_fin library

        self.orig_df = df.copy() # the original dataframe itself
        self.save_orig_df()

        # add date as a column
        if "date" not in df.columns:
            df["date"] = df.index
        return(df)

    def save_orig_df(self): # save the original dataframe
        date_now = time.strftime("%Y-%m-%d")
        self.ticker_data_filename = os.path.join("data", "%s_%s.csv" % (self.ticker, date_now))
        self.orig_df.to_csv(self.ticker_data_filename)

    # enhance each daily data with its history whose length is n_steps
    def preprocess_data(self, scaler = preprocessing.StandardScaler(), lookup_step=1, n_steps=50):
        """
        Params:
            scaler : preprocessing scaler, default is preprocessing.StandardScaler()
            # See https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing
            lookup_step (int): the future lookup step to predict, default is 1 (e.g next day)
            n_steps (int): the historical sequence length (i.e window size) used to predict, default is 50
        """
        self.scaler = scaler
        self.lookup_step = lookup_step
        self.n_steps = n_steps

        # scale the data (prices)
        for column in self.feature_columns:
            self.df[column] = self.scaler.fit_transform(np.expand_dims(self.df[column].values, axis=1))

        df = self.df.copy()

        # add 'future' column
        self.add_future(df)

        # convert each day data to a historical "n_steps"s data
        sequence_data = []
        sequences = deque(maxlen=self.n_steps)
        for entry, target in zip(df[self.feature_columns + ["date"]].values, df['future'].values):
            sequences.append(entry)
            if len(sequences) == self.n_steps:
                sequence_data.append([np.array(sequences), target])

        # get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence
        # for instance, if n_steps=50 and lookup_step=10, last_sequence should be of 60 (that is 50+10) length
        # this last_sequence will be used to predict future stock prices that are not available in the dataset
        last_sequence_list = list([s[:len(self.feature_columns)] for s in sequences]) + list(self.last_sequence)
        self.last_sequence = np.array(last_sequence_list).astype(np.float32)

        # construct the X's and y's
        X, y = [], []
        for seq, target in sequence_data:
            X.append(seq) # shape(seq)= (n_steps) x (# of feature_colums +1)
            y.append(target)
        # convert to numpy arrays
        self.X = np.array(X) # array of 'seq'
        self.y = np.array(y) # array of 'target'

    def add_future(self, df):
        # add future as a column
        df['future'] = df['adjclose'].shift(-self.lookup_step)
        # last `lookup_step` columns contains NaN in future column
        # save them as "self.last_sequence" before droping NaNs (the value of 'future' at the last index is 'NaN')
        self.last_sequence = np.array(df[self.feature_columns].tail(self.lookup_step)) # exclude 'date' field
        df.dropna(inplace=True) # drop NaNs

前半は，yahoo_fin からのデータの読み込みです．

self.future_columns には，データのカラムの中から，特徴量として利用するカラムのリストを指定しています．
メソッド load_df は，読み込んだデータを加工せずに，save_orig_df を使ってファイルに格納してます．
このとき，格納するフォルダ "./data" は main を起動するときに，（必要ならば）作成するようにします．

その後，インデックスとなっている日付情報を，新しいカラム "date" に格納します．

前半で読み込んだデータは，「日数 $\times$ レコード（カラムの集合）」の2次元の構造を持っています．
これを，この各レコードに，予め決めた日数（n_steps) の過去データを含めることによって，
「日数 $\times$ ヒストリー $\times$ レコード（カラムの集合）」
の3次元データを作成するのが，SPData の後半の仕事です．

後半は，メソッド preprocess_data で，これはデータ構築に用いられます．

ここでは副作用を避けるため，まずはデータをコピーします．

続いてスケーリングです．
学習に使うデータは，各カラム毎にスケールが違うと，問題が生じます．
そのため，

1. 平均0，標準偏差1に標準化する
1. 最大値を1，最小値を0となるように線形変換する

などのスケーリングをする必要があります．
メソッド "scale_data" は，sklearn.preprocessing を使って，これを行います．



予測すべき日付の価格を格納したカラム "future" をメソッドadd_future を使って追加します．
このとき，引数の lookup_step で，何日先を予測するかを指定します．
なお，"future" カラムの価格は教師データとして使われます．

次は，本命の3次元データの作成です．

リスト sequence_data
には，
[ n_steps日分の特徴量カラムの全体, 教師データ ]
のペアを「日数 - n_steps」分格納し，
それを Numpy配列にした結果を
self.X と self.y としています．

**課題**.
scaler の使用方法や種類を[マニュアル](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)で調べよ．

では，ここまでの部分を動かして，実際に，self.X と self.y がどのような形をしているか見てみましょう．

まずは，乱数の種の初期化と，必要なフォルダの生成を行う補助関数を定義します．

In [ ]:
### Miscellaneous Functions

def set_seeds(): # set seeds, so we can get the same results after rerunning several times
    np.random.seed(314)

def create_folders(): # create folders if they do not exist
    if not os.path.isdir("data"):
        os.mkdir("data")

では，走らせてみましょう．

In [ ]:
if __name__ == '__main__':

    set_seeds()
    create_folders()

    ### load the data ###

    #ticker = "GOOGL"
    #ticker = "AAPL"
    ticker = "META" # ex-FB
    #ticker = "AMZN"
    data = SPData(ticker)

    # for scaler, see https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing
    SCALER = preprocessing.MinMaxScaler()   # fit the data within [0,1]
    #SCALER = preprocessing.StandardScaler()    # normalize the data
    data.preprocess_data(scaler=SCALER, lookup_step=15, n_steps=50)

    print('data.X=', data.X)
    print('data.y=', data.y)

data.X が3次元データになっているのがわかるでしょう．
また数値が scaler によって基準化されている点にも注意してください．

クラス SPData で作られた
data.X, data.y
を材料にして，訓練用データとテスト用データを構築するのが，
つぎのクラス SPDataConstructor です．

In [ ]:
class SPDataConstructor(object):
    def __init__(self, data):
        self.data = data

    def construct(self, test_size=0.2):
        """
        Params:
            test_size (float): ratio for test data, default is 0.2 (20% testing data)
        """
        self.test_size = test_size

        # split the dataset into training & testing sets by date (not randomly splitting)
        train_samples = int((1 - self.test_size) * len(data.X)) # number of in-samples
        X_train = data.X[:train_samples] # shape(X_train)= (train_samples) x ((n_steps) x (# of feature_colums +1)
        self.y_train = data.y[:train_samples]
        X_test = data.X[train_samples:]
        self.y_test = data.y[train_samples:]

        self.dates = X_test[:, -1, -1] # get the list of test set dates (the last days of each sequence)
        # retrieve test features from the original dataframe
        self.test_df = self.data.orig_df.loc[self.dates]
        # remove duplicated dates in the testing dataframe
        self.test_df = self.test_df[~self.test_df.index.duplicated(keep='first')]
        # remove 'date' fields from the training/testing sets & convert to float32
        self.X_train = X_train[:, :, :len(self.data.feature_columns)].astype(np.float32)
        self.X_test = X_test[:, :, :len(self.data.feature_columns)].astype(np.float32)

        return((self.X_train, self.X_test, self.y_train, self.y_test))

ここで行うデータ構成方法としては，

1. データ全体を前半と後半に分け，それぞれを訓練用とテスト用に使う
1. 最初の1年分を訓練用としそれに続く1ヶ月をテスト用とする．さらに，一月後ろにずらしてつぎの1年分の訓練用データとそれに続く1ヶ月のテスト用データを構築．以下，これを繰り返す．

のように，様々な手法が考えられます．
ここでは，データ全体の前
1- test_size $\in[0, 1]$ を訓練データに，
残りをテスト・データとしています．

**課題**
X_train, y_train, X_test, y_test を Numpy array で実装されている．これは，ビッグデータやリアルタイム・データを扱う場合には利用できない．(何故か？) そこでこれら4つのオブジェクトを，[generator](https://wiki.python.org/moin/Generators) として実装してみよ．

**2 ナイーブな予測**

この節では，機械学習を使わずに，lookup_step日後の株価を予測するナイーブな方法を考えます．

まずは，
前節で構築した data_c.X_test と data_c.y_test
の内容を見てみましょう．

In [ ]:
if __name__ == '__main__':
    data_c = SPDataConstructor(data)
    data_c.construct(test_size=0.2)

    print('X_test=')
    print(data_c.X_test)
    print('X_test[-2]=', data_c.X_test[-2])
    print('X_test[-1]=', data_c.X_test[-1])
    pred_p = data_c.X_test[:, -1, 0]
    print('pred_p=', pred_p)
    print('y_test=', data_c.y_test)

ここで pred_p は，X_test の各日付の最終日の終値を表しています．
ナイーブな予測として，この pred_p の値を lookup_step日後の株価予測値とします．
そしてその予測値と y_test の値との差の絶対値の平均値，
すなわち Mean Absolute Error (MAE) を計算し，予測精度を評価します．

このナイーブ予測評価を行うクラス SPNaiveModel を以下に示します．

In [ ]:
import matplotlib.pyplot as plt

class SPNaiveModel(object):
    def __init__(self, data_c):
        """
        Params:
            data_c (SPDataConstructor): the data you are anlyzing
        """
        self.data_c = data_c
        self.data = data_c.data

        self.pred_p = self.data_c.X_test[:, -1, 0]
        self.true_p = self.data_c.y_test
        self.mae, self.mae_scaled = self.get_mae(self.true_p, self.pred_p)

    def examine(self):
        self.add_true_pred_p(self.true_p, self.pred_p)
        self.plot_true_pred_comparison()
        print('naive_MAE=', self.mae)
        print("naive_MAE_scaled=", self.mae_scaled)

    def get_mae(self, targets, preds): # compute MAE
        """
        Params:
            targets: true values
            preds: predicted values
        """
        batch_maes = []
        for setp in range(len(targets)):
            mae = np.mean(np.abs(preds - targets))
            batch_maes.append(mae)
        mae_scaled = np.mean(batch_maes)
        mae = self.data.scaler.inverse_transform([[mae_scaled]])[0][0]
        return([mae, mae_scaled])

    def add_true_pred_p(self, y_test, y_pred):
        """
        Params:
            y_test: array of true values
            y_pred: array of predicted values
        """
        test_df = self.data_c.test_df
        # add true future prices to the dataframe
        test_df["true_adjclose_%d" % data.lookup_step] = y_test
        # add naively-predicted future prices to the dataframe
        test_df["adjclose_naive"] = self.data.scaler.inverse_transform([y_pred])[0]

        # sort the dataframe by date
        test_df.sort_index(inplace=True)

        self.true_p = test_df["true_adjclose_%d" % data.lookup_step]
        self.naive_pred_p =  test_df["adjclose_naive"]
        return(test_df)

    # plot true close price along with predicted close price
    def plot_true_pred_comparison(self):
        f = plt.figure()
        f.set_size_inches(10, 7)
        plt.plot(self.true_p, label="Actual Price")
        dates = self.true_p.index
        plt.plot(dates, self.pred_p, label="Naively predicted Price")
        plt.title("Naive prediction(%s)" % self.data.ticker)
        plt.xlabel("days")
        plt.ylabel("price")
        plt.legend(loc='upper left', borderaxespad=3.0, fontsize=9)
        plt.show()

X_test の最終日の終値を pred_p に，
また y_test の値を真値 true_p として，
MAEを計算し，グラフに表示しています．

実行結果は以下のとおりです．

In [ ]:
if __name__ == '__main__':
    ### construct a non-ML model ###
    naive_model = SPNaiveModel(data_c)
    naive_model.examine()

上のグラフは scaled value で表示されていますが，n_steps 日ずれて予測値がプロットされているのがわかるでしょう．

一方，scaled MAEの 0.048 という値は，かなりよくて，これを打ち負かすのは，至難の業ということがいずれわかります．

**課題**
上記のグラフを unscaled の値で表示してみよ．

**3 深層学習による予測**

この節以降では，Keras の深層学習ライブラリを利用して，前節のナイーブ予測を打ち負かす株価予測に挑戦します．

前節までにトレーニングで利用した X_train や y_train は日付順に並んだデータでした．
しかし，ともするとこの日付順という暗黙の情報は学習に不利に働くことがあります．
このため，データを日付でシャッフルすることで，予測精度を向上させることができるかもしれません．

そこで，先に定義したクラス SPDataConstructor のコンストラクタの引数にブール値をとる shuffle を追加します．

In [ ]:
import random

class SPDataConstructor(object):
    def __init__(self, data):
        self.data = data

    def construct(self, shuffle=True, test_size=0.2):
        """
        Params:
            shuffle (bool): whether to shuffle the dataset (both training & testing), default is True
            test_size (float): ratio for test data, default is 0.2 (20% testing data)
        """
        self.shuffle = shuffle
        self.test_size = test_size

        # split the dataset into training & testing sets by date (not randomly splitting)
        train_samples = int((1 - self.test_size) * len(data.X)) # number of in-samples
        X_train = data.X[:train_samples] # shape(X_train)= (train_samples) x ((n_steps) x (# of feature_colums +1)
        self.y_train = data.y[:train_samples]
        X_test = data.X[train_samples:]
        self.y_test = data.y[train_samples:]
        if self.shuffle:
            self.shuffle_database(X_train, self.y_train, X_test, self.y_test)

        self.dates = X_test[:, -1, -1] # get the list of test set dates (the last days of each sequence)
        # retrieve test features from the original dataframe
        self.test_df = self.data.orig_df.loc[self.dates]
        # remove duplicated dates in the testing dataframe
        self.test_df = self.test_df[~self.test_df.index.duplicated(keep='first')]
        # remove 'date' fields from the training/testing sets & convert to float32
        self.X_train = X_train[:, :, :len(self.data.feature_columns)].astype(np.float32)
        self.X_test = X_test[:, :, :len(self.data.feature_columns)].astype(np.float32)

        return((self.X_train, self.X_test, self.y_train, self.y_test))

    def shuffle_database(self, X_train, y_train, X_test, y_test):
        state = np.random.get_state()
        np.random.shuffle(X_train)
        np.random.set_state(state)
        np.random.shuffle(y_train)
        state = np.random.get_state()
        np.random.shuffle(X_test)
        np.random.set_state(state)
        np.random.shuffle(y_test)

ではいよいよ，TensorFlow2 と Keras を使うことにします．
まずは，関係するライブラリを import します．

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

次にモデルを実装するクラス SPModel を導入します．

In [ ]:
# stock price model constructed with a recurrent neural network
class SPModel(object):
    def __init__(self, data_c, loss=losses.MeanAbsoluteError(), optimizer=optimizers.RMSprop()):
        """
        Params:
            data_c (SPDataConstructor): the data you are anlyzing.
            loss (loss function): see https://keras.io/api/losses/
            optimizer (optimizer object): see https://keras.io/api/optimizers/
        """

        self.data_c = data_c
        self.data = data_c.data
        self.loss = loss
        self.optimizer = optimizer

    # build and compile model.
    def build_model(self, metrics=metrics.Accuracy()):
        """
        Params:
            matrics (matrics object): see https://keras.io/api/metrics/
        """
        # model name to save, making it as unique as possible based on parameters
        date_now = time.strftime("%Y-%m-%d")
        shuffle_str = "sh-%d" % data_c.shuffle
        self.name = "%s_%s-%s" % (date_now, data.ticker, shuffle_str)
        self.model = models.Sequential()
        self.build()
        self.model.compile(loss=self.loss, metrics=metrics, optimizer=self.optimizer)

    def build(self): # virtual method
        pass

    # fit model.
    def fit(self, batch_size, epochs):
        """
        Params:
            batch (int): the size of batch.
            epochs (int): the number of epochs.
        """
        # some tensorflow callbacks
        # see https://keras.io/api/callbacks/
        # train the model and save the weights whenever we see
        # a new optimal model using ModelCheckpoint
        checkpointer = ModelCheckpoint(os.path.join("results", self.name + ".h5"), save_weights_only=True, save_best_only=True, verbose=1)
        tensorboard = TensorBoard(log_dir=os.path.join("logs", self.name))
        #tensorboard --logdir="logs"
        self.history = self.model.fit(self.data_c.X_train, self.data_c.y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(self.data_c.X_test, self.data_c.y_test),
                    callbacks=[checkpointer, tensorboard],
                    verbose=1)
        self.loss = self.history.history['loss']
        self.val_loss = self.history.history['val_loss']
        return(self.history)

SPModel のコンストラクタの引数は，

1. 先に作った DataConstructor のインスタンス
1. loss関数
1. optimizer

の3つで，その中身は，これらをデータメンバとして登録するだけです．

モデルを構築しコンパイルするメソッドは
build_model
です．このメソッドは metric関数を引数として持ちます．
metric関数は loss関数と似ていますが，訓練の際ではなくコンパイル時に使われるところが異なります．

実際のモデル構成は
build_model
のなかから呼ぶメソッド build で行います．
しかし SPModel ではこれは仮想メソッドとなっていて，実装は派生クラスで行うことになります．

モデル構成が終わったあと，コンパイルしています．

訓練は fit メソッドで行います．
このメソッドは
batchサイズと
繰り返し数 epochs
を引数として持ちます．

中身は Keras の model.fit を呼ぶところがだけが本質的ですが，
中間結果を
resultsフォルダと
logsフォルダに
毎epoch格納するためのコールバックを登録しています．
また，fit が終わったあと，
in-sample のエラー (val_loss)
と
out-of-sample のエラー (loss)
をデータメンバとして保持します．

**3.1 Flatten**

それでは SPModel の build メソッドを具体化した最初の例として，
SPModel から派生させたクラス
SPFlatten
を作成しましょう．
深層学習の最初の層を
Flatten層
にするこの例は，数ある深層学習モデルのベースラインとなるものです．

In [ ]:
class SPFlatten(SPModel):
    def __init__(self, data_c, loss=losses.MeanAbsoluteError(), optimizer=optimizers.RMSprop(), neurons=32, activation=activations.sigmoid):
        """
        Params:
            data_c (SPDataConstructor): the data you are anlyzing
            loss (loss function): see https://keras.io/api/losses/
            optimizer (optimizer object): see https://keras.io/api/optimizers/
            neurons (int): the number of artificial neurons per each layer, default is 32
            activation (activation object): see https://keras.io/ja/activations/
        """
        super().__init__(data_c, loss, optimizer)

        self.neurons = neurons
        self.activation = activation

    def build(self):
        layer = layers.Flatten
        self.name += self.name + f"-{self.loss.name}-{self.activation.__name__}-{self.optimizer.name}-{layer.__name__}-seq-{data.n_steps}-step-{data.lookup_step}-neurons-{self.neurons}"

        n_features = len(data.feature_columns)
        self.model.add(layer(batch_input_shape=(None, data.n_steps, n_features)))
        self.model.add(layers.Dense(self.neurons))

        self.model.add(layers.Dense(1, activation=self.activation))

SPFlatten
のコンストラクタの引数は，
SPModel
のそれに加えて
ニューロン数 neurons
と
活性化関数 activation
を持ちます．

build メソッドで行う
モデル構成は，入力層と出力層の間に中間層が1つある3層からなっています．

入力層で使われている Flatten層は，つぎつぎと入力される batch データを独立なデータと捉えて学習させます．
中間層と出力層は
指定されたニューロン数と活性関数をそれぞれ使った
全結合層(dense層)
を用いています．

結果を格納するファイルのファイル名
self.name
の指定で用いられている
loss.name, activation.\__name__, optimaizer._name
は，それぞれの名称を取り出すときに使うデータメンバが異なる点に注意してください．
こうした細かい情報は API のドキュメントにはでていないことがあり，
そういう場合は，GitHub にあるKerasの[ソースコード](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/python/keras)で確認する必要があります．

**課題**
[GitHub](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/python/keras) を探索して，上で示した名称のデータメンバのいくつかが，どこで決まっているかを確認せよ．

SPModel
(の派生クラス)
で作ったモデルの予測精度を評価するためのクラス
SPPredict を用意します．

In [ ]:
# predict and evaluate the constructed model
class SPPredict(object):
    def __init__(self, naive_model, model):
        """
        Params:
            naive_model (SPNaiveModel): the naive data you are refering
            model (SPModel): the data you are anlyzing
        """
        self.naive_model = naive_model
        self.model = model
        self.data_c = self.model.data_c
        self.data = self.model.data

    def examine(self):
        self.load_weights()
        self.get_final_df() # get the final dataframe for the testing set
        self.evaluate() # evaluate the model
        self.print_metrics() # printing metrics
        self.plot_true_pred_comparison() # plot true/predicted prices graph
        self.plot_epochs_mae() # plot MAE graph
        self.save_final_df() # save the final dataframe to csv-results folder

    def load_weights(self):
        # load optimal model weights from results folder
        model_path = os.path.join("results", self.model.name) + ".h5"
        self.model.model.load_weights(model_path)

    def get_final_df(self):
        """
        This function takes the `model` and `data` dict to
        construct a final dataframe that includes the features along
        with true and predicted prices of the testing dataset
        """

        X_test = self.data_c.X_test
        y_test = self.data_c.y_test
        # perform prediction and get prices
        y_pred = self.model.model.predict(X_test)
        y_test_exp = np.expand_dims(y_test, axis=0)
        self.mae, self.mae_scaled = self.naive_model.get_mae(y_test_exp, y_pred)
        y_test = np.squeeze(self.data.scaler.inverse_transform(y_test_exp))
        y_pred = np.squeeze(self.data.scaler.inverse_transform(y_pred))

        self.final_df = self.add_true_pred_p(y_test, y_pred)

    def evaluate(self): # evaluate the model
        loss, mae = self.model.model.evaluate(self.data_c.X_test, self.data_c.y_test, verbose=0)
        self.loss = loss
        # calculate the mean absolute error (inverse scaling)
        self.mean_absolute_error = mae

    def add_true_pred_p(self, y_test, y_pred):
        test_df = self.data_c.test_df
        # add true future prices to the dataframe
        test_df["true_adjclose_%d" % data.lookup_step] = y_test
        # add predicted future prices to the dataframe
        test_df["adjclose_%d" % data.lookup_step] = y_pred
        # add naively-predicted future prices to the dataframe
        test_df["adjclose_naive"] = self.data.scaler.inverse_transform([self.data_c.X_test[:,-1,0]])[0]

        # sort the dataframe by date
        test_df.sort_index(inplace=True)

        self.true_p = test_df["true_adjclose_%d" % data.lookup_step]
        self.pred_p = test_df["adjclose_%d" % data.lookup_step]
        self.naive_pred_p =  test_df["adjclose_naive"]
        return(test_df)

    # plot true close price along with predicted close price
    def plot_true_pred_comparison(self):
        f = plt.figure()
        f.set_size_inches(10, 7)
        plt.plot(self.true_p, label="Actual Price")
        plt.plot(self.pred_p, label="Predicted Price")
        plt.plot(self.naive_pred_p, label="Naively predicted Price")
        plt.title("Prediction(%s)" % self.data.ticker)
        plt.xlabel("days")
        plt.ylabel("price")
        plt.legend(loc='upper left', borderaxespad=3.0, fontsize=9)
        plt.show()

    # plot true close price along with predicted close price
    def plot_epochs_mae(self):
        f = plt.figure()
        f.set_size_inches(10, 7)
        max_epochs = len(self.model.loss)
        epochs = range(1, max_epochs + 1)
        plt.plot(epochs, self.model.loss, 'bo', label="Training loss")
        plt.plot(epochs, self.model.val_loss, 'b', label="Validation loss")
        plt.hlines([self.naive_model.mae_scaled], 0, max_epochs, "green", linestyles='dashed', label='non-ML baseline')
        plt.title('Training and validation loss')
        plt.xlabel("epochs")
        plt.ylabel("mae")
        plt.legend(loc='upper right', borderaxespad=3.0, fontsize=9)
        plt.show()

    # print metrics
    def print_metrics(self):
        print("model.loss=", self.model.loss)
        print("loss:", self.loss)
        print("MAE:", self.mae)
        print("MAE_scaled:", self.mae_scaled)
        print('naive_MAE=', self.naive_model.mae)
        print("naive_MAE_scaled=", self.naive_model.mae_scaled)

    # save the final dataframe to csv-results folder
    def save_final_df(self):
        csv_filename = os.path.join("csv-results", self.model.name + ".csv")
        self.final_df.to_csv(csv_filename)

SPPredict のコンストラクタは，
naiveモデル (非機械学習モデル)
のインスタンス
naive_model
と
SPModel (機械学習モデル)
のインスタンス
model
を引数として持ちます．

メソッド
examine
のなかで，個々の評価用サブメソッドを実行します．

最初は，results フォルダから訓練済みのパラメタを読み込む
load_weights メソッド．
そこからテストデータとその予測値を含んだデータベースを復元し
(get_final_df)，
各種指標と図表を表示させています．

以上のコードを走らせるために，Naive model でも使った乱数シードの設定関数とフォルダの設定関数を以下のようにアップグレードします．

In [ ]:
### Miscellaneous Functions

def set_seeds(): # set seeds, so we can get the same results after rerunning several times
    np.random.seed(314)
    random.seed(314)
    tf.random.set_seed(314)


def create_folders(): # create folders if they do not exist
    if not os.path.isdir("data"):
        os.mkdir("data")
    if not os.path.isdir("results"):
        os.mkdir("results")
    if not os.path.isdir("logs"):
        os.mkdir("logs")
    if not os.path.isdir("csv-results"):
        os.mkdir("csv-results")

実行するためのメインプログラムは以下のようになります．

In [ ]:
if __name__ == '__main__':

    set_seeds()
    create_folders()

    ### load the data ###

    ticker = "GOOGL"
    #ticker = "AAPL"
    #ticker = "META"  # ex-FB
    #ticker = "AMZN"
    data = SPData(ticker)

    #data.set_params(shuffle=True, test_size=0.2)

    # for scaler, see https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing
    SCALER = preprocessing.MinMaxScaler()   # fit the data within [0,1]
    #SCALER = preprocessing.StandardScaler()    # normalize the data
    data.preprocess_data(scaler=SCALER, lookup_step=15, n_steps=50)
    data_c = SPDataConstructor(data)
    data_c.construct(shuffle=True, test_size=0.2)

    ### construct a non-ML model ###
    naive_model = SPNaiveModel(data_c)

    ### construct the model ###

    # for loss function, see https://keras.io/api/losses/
    LOSS = losses.MeanAbsoluteError()
    #LOSS = losses.Huber()
    #LOSS = losses.BinaryCrossentropy()

    # for optimizer, see https://keras.io/api/optimizers/
    #OPTIMIZER = optimizers.RMSprop()
    OPTIMIZER = optimizers.Adam()

    ## for activation function, see https://keras.io/ja/activations/
    # for activation function, see https://keras.io/api/layers/activations/
    #ACTIVATION = activations.relu
    #ACTIVATION = activations.sigmoid
    ACTIVATION = activations.linear

    model = SPFlatten(data_c, loss=LOSS, optimizer=OPTIMIZER, neurons=32, activation=ACTIVATION)

    # for metrics, see https://keras.io/api/metrics/
    #METRICS = metrics.Accuracy()
    METRICS = metrics.MeanAbsoluteError()
    model.build_model(METRICS)

    history = model.fit(batch_size=64, epochs=10)

    orig_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(val_loss) + 1)
    print('epochs=', epochs)
    print('loss=', orig_loss)
    print('val_loss=', val_loss)

    ### examine the trained model ###

    pred = SPPredict(naive_model, model)
    pred.examine()

一番下の図の中の緑の破線は，naiveモデルでの mean abosolute error を表しています．これよりも下にあれば，naive モデルより予測精度の良いモデルということになります．

in-sample の trading loss こそ下回っていますが，
out-of-sample の validation loss は破線の上側にあり，
naiveモデルに負けていることがわかります．

**3.2 GRU**

Flatten層では時間情報を潰してしまっていましたが，
これを直前の batch data の処理後のデータを参照することによって
過去のデータに依存した処理を行えるようにした
recurrent neural network (RNN)
に置き換えることによって，予測精度を向上させることを考えてみます．

ここでは，RNN のひとつである GRU層を使ったクラス
SPGRU を実装してみます．

In [ ]:
class SPGRU(SPModel):
    def __init__(self, data_c, loss=losses.MeanAbsoluteError(), optimizer=optimizers.RMSprop(), neurons=32, activation=activations.sigmoid):
        """
        Params:
            data_c (SPDataConstructor): the data you are anlyzing
            loss (loss function): see https://keras.io/api/losses/
            optimizer (optimizer object): see https://keras.io/api/optimizers/
            neurons (int): the number of artificial neurons per each layer, default is 32
            activation (activation object): see https://keras.io/ja/activations/
        """
        super().__init__(data_c, loss, optimizer)

        self.neurons = neurons
        self.activation = activation

    def build(self):
        layer = layers.GRU
        self.name += self.name + f"-{self.loss.name}-{self.activation.__name__}-{self.optimizer.name}-{layer.__name__}-seq-{data.n_steps}-step-{data.lookup_step}-neurons-{self.neurons}"

        n_features = len(data.feature_columns)
        self.model.add(layer(self.neurons,
                            dropout=0.2,
                            recurrent_dropout=0.2,
                            return_sequences=False,
                            batch_input_shape=(None, data.n_steps, n_features)))
        #self.model.add(layer(self.neurons, return_sequences=False))
        self.model.add(layers.Dense(1, activation=self.activation))

SPGRU のコンストラクタは，SPFlatten のそれとまったく同じで，
build メソッドの中で構築されるモデルのみが異なります．

今回は2層にしていて，入力層にGRU層を用いています．
ここで
dropout
と
recurrent_dropout
は過学習を低減するためのパラメタです．

GRUモデルを走らせるためのメインプログラムは，Flattenモデルのそれを1行だけ変更したものです．

In [ ]:
if __name__ == '__main__':

    set_seeds()
    create_folders()

    ### load the data ###

    #ticker = "GOOGL"
    #ticker = "AAPL"
    ticker = "META" # ex-FB
    #ticker = "AMZN"
    data = SPData(ticker)

    #data.set_params(shuffle=True, test_size=0.2)

    # for scaler, see https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing
    SCALER = preprocessing.MinMaxScaler()   # fit the data within [0,1]
    #SCALER = preprocessing.StandardScaler()    # normalize the data
    data.preprocess_data(scaler=SCALER, lookup_step=15, n_steps=50)
    data_c = SPDataConstructor(data)
    data_c.construct(shuffle=True, test_size=0.2)

    ### construct a non-ML model ###
    naive_model = SPNaiveModel(data_c)

    ### construct the model ###

    # for loss function, see https://keras.io/api/losses/
    LOSS = losses.MeanAbsoluteError()
    #LOSS = losses.Huber()
    #LOSS = losses.BinaryCrossentropy()

    # for optimizer, see https://keras.io/api/optimizers/
    #OPTIMIZER = optimizers.RMSprop()
    OPTIMIZER = optimizers.Adam()

    ## for activation function, see https://keras.io/ja/activations/
    # for activation function, see https://keras.io/api/layers/activations/
    #ACTIVATION = activations.relu
    #ACTIVATION = activations.sigmoid
    ACTIVATION = activations.linear

    model = SPGRU(data_c, loss=LOSS, optimizer=OPTIMIZER, neurons=32, activation=ACTIVATION)

    # for metrics, see https://keras.io/api/metrics/
    #METRICS = metrics.Accuracy()
    METRICS = metrics.MeanAbsoluteError()
    model.build_model(METRICS)

    history = model.fit(batch_size=64, epochs=10)

    orig_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(val_loss) + 1)
    print('epochs=', epochs)
    print('loss=', orig_loss)
    print('val_loss=', val_loss)

    ### examine the trained model ###

    pred = SPPredict(naive_model, model)
    pred.examine()

残念ながら，このモデルでも，naiveモデルを打ち負かすことはできていません．

**課題**
工夫して，より高精度な予測を行うモデルを探せ．

例えば，以下のような点の変更が考えられるが，これらがすべてではない．

1. [optimizer](https://keras.io/api/optimizers/) (RMSprop, Adam 等）
1. [metrics](https://keras.io/api/metrics/) (Accuracy, MeanAbsoluteError 等)
1. [layer](https://keras.io/ja/layers/recurrent/)(SimpleRNN, GRU, LSTM 等)
1. [loss関数](https://keras.io/api/losses/)(MAE, Huber, Crossentropy 等)
1. [activation関数](https://keras.io/api/layers/activations/)(relu, sigmoid, linear 等)
1. dropout および recurrent_dropout
1. neuron数
1. layerの段数と構成
1. epoch数
1. shuffle のあるなし
1. [scaler](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing) (MinMaxScaler, StandardScaler 等）

また上に挙げた変更可能なパーツの多くは，独自に改良したコードに置き換えることも可能である．この場合は，[GitHub](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/python/keras) を参考にされたい．

**課題**
上記課題を，複数の銘柄 (ticker）で行い，見つけたモデルの堅牢性，つまり，一般的にどの程度利用可能かを検討せよ．

**課題**
SPDataConsturctor は，全データから1組だけ訓練データとテストデータをとっているが，現実のように取引を継続して行う場合には，異なる抽出方法で評価する必要があるだろう．
そうした評価方法を考え，実装せよ．

**課題**
baseline (naive model) よりも大きく精度が高い予測を行おうとするには，今回の問題のセッティングのどこを再考すればよいか議論せよ．